## Read Files (failed)

### method 1 (chunking)

In [3]:
import rasterio
from rasterio.plot import show

# Path to the GeoTIFF file
tiff_file_path = "D:/Yehmh/test_py/202301/P00073_transect_234/202301P00073_RGB_transparent_mosaic_group1.tif"

# Specify the block size (e.g., 256x256)
blockxsize = 1
blockysize = 1

# Open the GeoTIFF file with chunking
with rasterio.open(tiff_file_path, blockxsize=blockxsize, blockysize=blockysize) as src:
    # Read raster data
    raster_data = src.read()
    
    print("test")

    # Visualize the raster data
    # show(src, cmap='viridis')  # Adjust the colormap as needed


test


In [2]:
print("Raster data shape:", raster_data.shape)
print("Number of bands:", src.count)
print("Data type:", raster_data.dtype)

Raster data shape: (4, 52234, 45030)
Number of bands: 4
Data type: uint8


In [4]:
src.meta

{'driver': 'GTiff',
 'dtype': 'uint8',
 'nodata': None,
 'width': 45030,
 'height': 52234,
 'count': 4,
 'crs': CRS.from_epsg(3826),
 'transform': Affine(0.02985, 0.0, 299463.10392,
        0.0, -0.02985, 2632273.00498)}

### method 2 (gdal)

In [5]:
from osgeo import gdal

tif_file = gdal.Open("D:/Yehmh/test_py/202301/P00073_transect_234/202301P00073_RGB_transparent_mosaic_group1.tif")
geotransform = tif_file.GetGeoTransform()

# Get the geotransform information
xmin = geotransform[0] # TM2 (m)
ymax = geotransform[3] # TM2 (m)
res = geotransform[1] # meters/pixel

# Get the raster size
x_size = tif_file.RasterXSize # pixels
y_size = tif_file.RasterYSize # pixels
xlen = res * x_size # meters
ylen = res * y_size # meters


In [44]:
import os
import numpy as np

# Iterate over each pixel
for y in range(y_size):
    for x in range(x_size):
        # Calculate the coordinates of the pixel
        x_coord = xmin + x * res
        y_coord = ymax + y * res
        
        # Define the bounding box for the 1m x 1m area
        x_min_1m = x_coord
        y_max_1m = y_coord
        x_max_1m = x_coord + res
        y_min_1m = y_coord - res
        
        # Read the pixel value
        pixel_values = tif_file.ReadAsArray(x, y, 1, 1)
        # # Select a single value from the array
        # pixel_value = pixel_values[0][0]
        # # Convert the pixel value to a NumPy array
        # pixel_array = np.array([[pixel_value]], dtype=np.float32)

        # print(pixel_values.shape[0])
        print("x: ", x, ", y: ", y)
        
        # Create a new dataset for the 1m x 1m area
        driver = gdal.GetDriverByName('GTiff')
        out_tif = driver.Create(
            f"D:/Yehmh/test_py/202301/P00073_transect_234/test/{x_coord}_{y_coord}.tif",
            1, 1, 1, gdal.GDT_Float32
        )
        
        # Set the geotransform
        out_tif.SetGeoTransform((x_min_1m, res, 0, y_max_1m, 0, -res))
        
        # Write the pixel value to the new dataset
        # out_tif.GetRasterBand(1).WriteArray(pixel_values)
        for band_num in range(pixel_values.shape[0]):
            band_values = pixel_values[band_num]
            out_tif.GetRasterBand(band_num + 1).WriteArray(band_values)
        
        # Close the dataset
        out_tif = None

# Close the original TIFF file
tif_file = None

AttributeError: 'NoneType' object has no attribute 'ReadAsArray'

In [42]:
import os
import numpy as np
from osgeo import gdal
import math

# Open the original TIFF file
tif_file = gdal.Open("D:/Yehmh/test_py/202301/P00074_transect_1/202301P00074_RGB_transparent_mosaic_group1.tif")

# Get raster size and geotransform
x_size = tif_file.RasterXSize
y_size = tif_file.RasterYSize
geotransform = tif_file.GetGeoTransform()
xmin, ymax, resx, resy = geotransform[0], geotransform[3], geotransform[1], geotransform[5]

# Iterate over each pixel
for y in range(y_size):
    for x in range(x_size):
        # Calculate the coordinates of the pixel
        x_coord = xmin + x * resx
        y_coord = ymax + y * resy
        
        # Define the bounding box for the 1m x 1m area
        x_min_1m = x_coord
        y_max_1m = y_coord
        x_max_1m = x_coord + resx
        y_min_1m = y_coord - resy
        
        # Read the pixel values
        pixel_values = tif_file.ReadAsArray(x, y, 1, 1)
        
        # Create a new dataset for the 1m x 1m area
        driver = gdal.GetDriverByName('GTiff')
        out_tif = driver.Create(
            f"D:/Yehmh/test_py/202301/P00074_transect_1/{x_coord}_{y_coord}.tif",
            1, 1, tif_file.RasterCount, gdal.GDT_Float32
        )
        
        # Set the geotransform
        out_tif.SetGeoTransform((x_min_1m, resx, 0, y_max_1m, 0, resy))
        
        # Write the pixel values to the new dataset
        for band_num in range(tif_file.RasterCount):
            band_values = pixel_values[band_num]
            out_tif.GetRasterBand(band_num + 1).WriteArray(band_values)
        
        # Close the new dataset
        out_tif = None

# Close the original TIFF file
tif_file = None


AttributeError: 'NoneType' object has no attribute 'SetGeoTransform'

In [43]:
import os
import numpy as np
from osgeo import gdal

# Open the original TIFF file
tif_file = gdal.Open("D:/Yehmh/test_py/202301/P00073_transect_234/202301P00073_RGB_transparent_mosaic_group1.tif")

# Get raster size and geotransform
x_size = tif_file.RasterXSize
y_size = tif_file.RasterYSize
geotransform = tif_file.GetGeoTransform()
xmin, ymax, resx, resy = geotransform[0], geotransform[3], geotransform[1], geotransform[5]

# Define the size of the blocks
block_size = 1  # 1 meter by 1 meter

# Iterate over the bounding boxes of each 1m x 1m block
for y in np.arange(ymax, ymax - y_size * resy, -block_size):
    for x in np.arange(xmin, xmin + x_size * resx, block_size):
        # Calculate the coordinates of the block
        x_min_block, y_max_block = x, y
        x_max_block, y_min_block = x + block_size, y - block_size

        # Read the pixel values for the block
        pixel_values = tif_file.ReadAsArray(
            int((x - xmin) / resx), int((ymax - y) / resy), int(block_size / resx), int(block_size / resy))

        # Create a new dataset for the block
        driver = gdal.GetDriverByName('GTiff')
        out_tif = driver.Create(
            f"D:/Yehmh/test_py/202301/P00073_transect_234/test/{x_min_block}_{y_max_block}.tif",
            int(block_size / resx), int(block_size / resy), tif_file.RasterCount, gdal.GDT_Float32)

        # Set the geotransform
        out_tif.SetGeoTransform((x_min_block, resx, 0, y_max_block, 0, resy))

        # Write the pixel values to the new dataset
        for band_num in range(tif_file.RasterCount):
            band_values = pixel_values[band_num]
            out_tif.GetRasterBand(band_num + 1).WriteArray(band_values)

        # Close the new dataset
        out_tif = None

# Close the original TIFF file
tif_file = None


## GPS

### Extract GPS coordinates from .jpg

In [10]:
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS

def get_exif(filename):
    exif = Image.open(filename)._getexif()

    if exif is not None:
        # Create a copy of the keys before iterating
        keys = list(exif.keys())
        for key in keys:
            name = TAGS.get(key, key)
            exif[name] = exif.pop(key)

        if 'GPSInfo' in exif:
            # Create a copy of the GPSInfo keys before iterating
            gps_keys = list(exif['GPSInfo'].keys())
            for key in gps_keys:
                name = GPSTAGS.get(key, key)
                exif['GPSInfo'][name] = exif['GPSInfo'].pop(key)

    return exif

exif = get_exif('D:/Yehmh/test_py/DJI_0021_for_test.JPG')
print(exif)


{'GPSInfo': {'GPSVersionID': b'\x02\x03\x00\x00', 'GPSLatitudeRef': 'N', 'GPSLatitude': (23.0, 46.0, 40.6655), 'GPSLongitudeRef': 'E', 'GPSLongitude': (121.0, 29.0, 15.6631), 'GPSAltitudeRef': b'\x00', 'GPSAltitude': 110.397}, 'ResolutionUnit': 2, 'ExifOffset': 182, 'ImageDescription': 'DCIM\\100MEDIA\\DJI_0021.JPG', 'Make': 'DJI\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 'Model': 'FC6310\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 'Software': 'v01.07.1641\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 'Orientation': 1, 'DateTime': '2023:02:02 09:16:41', 'YCbCrPositioning': 1, 'XResolution': 72.0, 'YResolution': 72.0, 'XPComment': b'T\x00y\x00p\x00e\x00=\x00N\x00,\x00 \x00M\x00o\x00d\x00e\x00=\x00P\x00,\x00 \x00D\x00E\x00=\x00N\x00o\x00n\x00e\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

### traps GPS + which species

In [5]:
import pandas as pd
import twd97

# Read the CSV files into pandas DataFrames
seedtrapGPS_df = pd.read_csv("D:/Yehmh/test_py/seedtrapGPS.csv")  # Assuming the file name is "traps.csv"
# PlantsAroundTraps_df = pd.read_csv("D:/Yehmh/test_py/PlantsAroundTraps_ZF.csv")  # Assuming the file name is "species.csv"
PlantsAroundTraps_df = pd.read_csv("D:/Yehmh/test_py/PlantsAroundTraps_ZF.csv", encoding='latin1')


# Group species data by trap location and find the most common species in each trap
main_species_df = PlantsAroundTraps_df.groupby('Plot')['SpeciesID'].agg(lambda x: x.value_counts().idxmax()).reset_index()
# print(seedtrapGPS_df)

# Merge the main species data with trap locations
merged_df = pd.merge(seedtrapGPS_df, main_species_df, on='Plot', how='left')
# print(merged_df)

# Apply conversion function to each row of the DataFrame
merged_df['TWD97_X'], merged_df['TWD97_Y'] = zip(*merged_df.apply(lambda row: twd97.fromwgs84(row['latitude'], row['longitude']), axis=1))

# Display the merged DataFrame
print("\nMerged DataFrame:")
print(merged_df)

# Now, you can save the merged DataFrame to a new CSV file if needed
# merged_df.to_csv("traps_main_species_twd97.csv", index=False)



Merged DataFrame:
     Plot   latitude   longitude  SpeciesID        TWD97_X       TWD97_Y
0    T1S1  23.796614  121.491990        NaN  300132.554107  2.632586e+06
1    T1S2  23.796413  121.492345        5.0  300168.802512  2.632564e+06
2    T1S3  23.796279  121.492650        2.0  300199.920424  2.632549e+06
3    T1S4  23.796107  121.493068        1.0  300242.587235  2.632530e+06
4    T1S5  23.795898  121.493360        2.0  300272.450657  2.632507e+06
5    T1S6  23.795762  121.493709        5.0  300308.062925  2.632492e+06
6    T1S7  23.795618  121.494023        2.0  300340.102146  2.632476e+06
7    T1S8  23.795506  121.494466        2.0  300385.303754  2.632464e+06
8    T1S9  23.795326  121.494791        5.0  300418.406486  2.632444e+06
9   T1S10  23.795189  121.495153        7.0  300455.354170  2.632429e+06
10   T2S1  23.792507  121.492907        6.0  300227.566492  2.632131e+06
11   T2S2  23.792311  121.493176        6.0  300255.051336  2.632110e+06
12   T2S3  23.792181  121.493561

In [1]:
import pandas as pd
import twd97

# Read the CSV files into pandas DataFrames
seedtrapGPS_df = pd.read_csv("D:/Yehmh/test_py/seedtrapGPS.csv")
PlantsAroundTraps_df = pd.read_csv("D:/Yehmh/test_py/PlantsAroundTraps_ZF.csv", encoding='latin1')

# Group species data by trap location and find the most common species in each trap
main_species_df = PlantsAroundTraps_df.groupby('Plot')['SpeciesID'].agg(lambda x: x.value_counts().idxmax()).reset_index()

# Calculate the counts of each species in each plot
species_counts = PlantsAroundTraps_df.groupby(['Plot', 'SpeciesID']).size().reset_index(name='Count')
# print(species_counts)

# Calculate the total count of species in each plot
total_counts = species_counts.groupby('Plot')['Count'].sum().reset_index(name='TotalCount')
# print(total_counts)

# Merge total counts with species counts
species_counts = pd.merge(species_counts, total_counts, on='Plot', how='left')

# Calculate the percentage of the main species in each plot
species_counts['Percentage'] = (species_counts['Count'] / species_counts['TotalCount']) * 100
print(species_counts[species_counts['Percentage'] >= 80])



     Plot  SpeciesID  Count  TotalCount  Percentage
4    T1S3          2      1           1  100.000000
5    T1S4          1      4           4  100.000000
6    T1S5          2      1           1  100.000000
8    T1S6          5     11          13   84.615385
13   T1S8          2      8           9   88.888889
14   T1S9          5      2           2  100.000000
15   T2S1          6     17          17  100.000000
16  T2S10          5      9          11   81.818182
18   T2S2          6     20          20  100.000000
19   T2S3          6     22          22  100.000000
20   T2S4          6     19          19  100.000000
21   T2S5          6     25          25  100.000000
22   T2S6          6     21          21  100.000000
23   T2S7          6     20          20  100.000000
24   T2S8          6     13          13  100.000000
28   T3S1          4     30          30  100.000000
29  T3S10          6      6           6  100.000000
30   T3S2          4     29          29  100.000000
31   T3S3   

### GPS (10 meters around traps)

In [29]:
def points_around_center(center, distance):
    """
    Generate four points around the given center point, forming a square with sides aligned with the cardinal directions.

    Args:
        center (tuple): Tuple containing the latitude and longitude of the center point.
        distance (float): Distance in meters from the center point to each of the four points.

    Returns:
        List of tuples, each containing latitude and longitude of a point around the center.
    """
    lat, lon = center
    
    # Calculate latitude and longitude adjustments for north, south, east, and west points
    lat_adj = distance / 111111  # 1 degree of latitude is approximately 111111 meters
    lon_adj = distance / (111111 * (1 / abs((lat))))  # Longitude adjustment depends on latitude
    
    # Generate the four points
    north_point = (lat + lat_adj, lon)
    south_point = (lat - lat_adj, lon)
    east_point = (lat, lon + lon_adj)
    west_point = (lat, lon - lon_adj)
    
    return [north_point, south_point, east_point, west_point]

# Example usage
center_point = (40.7128, -74.0060)  # New York City
distance_meters = 10
points = points_around_center(center_point, distance_meters)

# Print the generated points
for i, point in enumerate(points):
    print(f"Point {i+1}: Latitude = {point[0]}, Longitude = {point[1]}")


Point 1: Latitude = 40.71289000009, Longitude = -74.006
Point 2: Latitude = 40.712709999910004, Longitude = -74.006
Point 3: Latitude = 40.7128, Longitude = -74.00233584433585
Point 4: Latitude = 40.7128, Longitude = -74.00966415566415


## Dataset

In [8]:
import os

base_folder = "D:/Yehmh/test_py/202301/P00074_transect_1/1m_1m/known/"

# Create folders T1S1 to T1S10
for i in range(1, 11):
    folder_name = f"T1S{i}"
    folder_path = os.path.join(base_folder, folder_name)
    os.makedirs(folder_path)

print("Folders created successfully!")



Folders created successfully!


In [ ]:
import os
import numpy as np
import pandas as pd
from osgeo import gdal
from osgeo import osr
import math

# Open the original TIFF file
folder_path = "D:/Yehmh/test_py/202301/P00074_transect_1/"
file_name = "202301P00074_RGB_transparent_mosaic_group1.tif"
tif_file = gdal.Open(os.path.join(folder_path, file_name))

# Read the CSV file
traps_info = pd.read_csv("traps_main_species_twd97.csv")
# Filter the DataFrame to include only rows where Plot starts with "T1S"
t1_info = traps_info[traps_info['Plot'].str.startswith("T1S")]

# Get the geotransform information
geotransform = tif_file.GetGeoTransform()
xmin = geotransform[0] # TM2 (m)
ymax = geotransform[3] # TM2 (m)
res = geotransform[1] # meters/pixel

# Get the raster size
x_size = tif_file.RasterXSize # pixels
y_size = tif_file.RasterYSize # pixels
xlen = res * x_size # meters
ylen = res * y_size # meters

# Define the size of the blocks
block_size = 1 # 1 meter by 1 meter

test_1 = 0
test_2 = 0

# Iterate over the bounding boxes of each 1m x 1m block
for y in np.arange(ymax, ymax - y_size * res, -block_size):
    for x in np.arange(xmin, xmin + x_size * res, block_size):
        # Calculate the coordinates of the block
        x_min_block, y_max_block = x, y
        x_max_block, y_min_block = x + block_size, y - block_size
        block_coord = [x + block_size/2, y - block_size/2]

        # Read the pixel values for the block
        pixel_values = tif_file.ReadAsArray(
            int((x - xmin) / res), int((ymax - y) / res), int(block_size / res), int(block_size / res))
        
        if pixel_values is None:
            # print("Error: Failed to read pixel values.", test_1)            
            # print(int(block_coord[0]), int(block_coord[1]), test_1)
            # test_1 = test_1 + 1
            continue  # Skip processing this block and continue to the next one

        # plt.figure()
        # plt.imshow(pixel_values[0], cmap='gray')

        # Find the closest plot
        min_distance = float('inf')
        closest_t1s = None
        for index, row in t1_info.iterrows():
            plot_name = row['Plot']
            plot_coord = [row['TWD97_X'], row['TWD97_Y']]
            distance = math.dist(plot_coord, block_coord)
            if distance < min_distance:
                min_distance = distance
                closest_plot = plot_name
            
        # if int(block_coord[0])%300130 < 10 and int(block_coord[1])%2632580 < 10:
        #     print(closest_plot, min_distance, int(block_coord[0]), int(block_coord[1]))

        # Move the file to the appropriate folder
        if min_distance < 10:  # If distance is less than 10 meters
            folder = os.path.join("known", closest_plot)
            print(closest_plot, min_distance, int(block_coord[0]), int(block_coord[1]))
        else:
            folder = "unknown"
            # print(min_distance)
            continue

        
        # Create a new dataset for the 1m x 1m area
        driver = gdal.GetDriverByName('GTiff')

        if driver is None:
            print("Error: Failed to get GDAL driver.")
            exit(1)        
        
        out_tif = driver.Create(
            os.path.join(folder_path, "1m_1m", folder, f"{int(block_coord[0])}_{int(block_coord[1])}.tif"),
            int(block_size/res), int(block_size/res), tif_file.RasterCount, gdal.GDT_Byte
        )

        if out_tif is None:
            # print("Error: Failed to create output TIFF dataset.")
            # exit(1)
            print("Error: Failed to create output TIFF dataset.", test_2)
            test_2 = test_2 + 1
            continue  # Skip processing this block and continue to the next one
        
        # Set the geotransform
        out_tif.SetGeoTransform((x_min_block, res, 0, y_max_block, 0, -res))
        srs = osr.SpatialReference()            # establish encoding
        srs.ImportFromEPSG(3826)                # TWD97 lat/long
        out_tif.SetProjection(srs.ExportToWkt()) # export coords to file

        # Print the dimensions of the array
        # print("Array dimensions:", pixel_values.shape)

        # Write the pixel values to the new dataset
        # for band_num in range(tif_file.RasterCount):
        #     band_values = pixel_values[band_num]
        #     print(band_num, band_values.shape)
        #     out_tif.GetRasterBand(band_num + 1).WriteArray(band_values)
        #     # out_tif.GetRasterBand(band_num + 1).WriteArray(pixel_values, 0, 0, x_size, y_size)

        for band_num in range(tif_file.RasterCount):
            band_values = pixel_values[band_num]
            out_tif.GetRasterBand(band_num + 1).WriteArray(band_values)
        
        # Set the metadata to include the block coordinates
        # metadata = f"x_min={x_min_block}, x_max={x_max_block}, y_min={y_min_block}, y_max={y_max_block}"
        # out_tif.SetMetadata({'Coordinates': metadata})
        
        # write to disk
        out_tif.FlushCache()

        # Close the new dataset
        out_tif = None

# Close the original TIFF file
tif_file = None